(final_template)=

# Final Project: Data Preprocessing

:::{epigraph}

-- Student name: Ngoc Linh Le
:::

The purpose of data process is to prepare a clean and focused data frame for the final project: Estimating the Causal Effect of High Short Video Usage on Academic Performance Among Elementary Students. 

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import ipywidgets as widgets

In [46]:
# import dataset
vid_df = pd.read_excel('~/comsc341cd.github.io/projects/final_project/final-data.xlsx')
vid_df

,1.No.,2.gender,3.school,4.grade,5.class,"6.The role of the ""filler""",7、What is your academic qualification?,8.Does he have his own cell phone?,9.Does he have his own tablet?,10.Parental Intervention (Time limit ),...,Subjective SVU8,Attention1,Attention2,Attention3,Attention4,Att Total,Academic：chinese,Academic：math,Academic：English,Academic：Total
0,1,1,1,2,3,2,6,2,1,NaN,...,1,73,16,18,56,163,99.0,78.0,99.0,276.0
1,2,1,1,2,3,2,5,2,2,NaN,...,2,114,11,13,79,217,100.0,95.0,100.0,295.0
2,3,2,1,2,3,2,5,2,2,1.0,...,1,69,33,16,66,184,100.0,89.0,100.0,289.0
3,4,2,1,5,2,2,5,2,2,1.0,...,0,74,25,27,96,222,78.0,86.0,80.0,244.0
4,5,1,1,2,3,2,5,2,1,1.0,...,3,53,5,8,40,106,95.0,94.0,90.0,279.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047,1082,2,2,2,2,2,4,2,2,NaN,...,5,67,15,20,1,103,60.0,48.5,89.0,197.5
1048,1083,2,2,2,2,2,5,2,2,NaN,...,3,46,2,8,22,78,66.0,54.0,83.0,203.0
1049,1084,2,2,1,5,2,6,2,1,NaN,...,2,61,25,22,64,172,98.0,100.0,93.0,291.0
1050,1085,1,2,3,4,2,4,2,1,NaN,...,3,59,16,13,66,154,86.0,115.0,99.0,300.0


In particular, there are the following variables in the dataset:

In [47]:
# Column names 
vid_df.columns

Index(['1.No.', '2.gender', '3.school', '4.grade', '5.class',
       '6.The role of the "filler"', '7、What is your academic qualification?',
       '8.Does he have his own cell phone?', '9.Does he have his own tablet?',
       '10.Parental Intervention (Time limit )',
       '10.Parental Intervention (Duration restrictions )',
       '10.Parental Intervention (content)',
       '10.Parental Intervention (No restrictions )',
       '10.Parental Intervention ( I can't control it anymore.)',
       '11.Is there any argument between parents and children on the use of electronic products?',
       '12.does your child enjoy watching short videos?',
       '13.Parental Intervention (Time limit )',
       '13.Parental Intervention (Duration restrictions )',
       '13.Parental Intervention (content)',
       '13.Parental Intervention (No restrictions )',
       '13.Parental Intervention ( I can't control it anymore.)',
       '14、Did you set "Teen Mode" on his short video app?',
       '15、How

Based on the study design, I selected the variables most relevant to estimating the causal effect of high short-video usage on academic performance. The following code (1) renames the raw survey columns into clear, analysis-ready variable names, and (2) filters the dataset to include only the key covariates and outcome measures used in the causal analysis: student demographics (gender, school type, grade), parent's education level, daily short-video usage duration, and total exam score.

In [49]:
# rename columns
vid_df = vid_df.rename(columns={
    '2.gender':'gender', # student's gender
    '3.school':'school', # student's school type 
    '4.grade':'grade', # student's grade
    '7、What is your academic qualification?': 'parent_edu', # parent's education level
    '15、How long do you spend on average to use short videos every day?':'ave_short_video_usage/day', # daily short-video usage duration
    'Academic：Total':'exam_score' # total exam score 
})

# filter the important columns to focus on
vid_df = vid_df.loc[:, ['gender', 'school', 'grade', 'parent_edu', 'ave_short_video_usage/day', 'exam_score']]

In [50]:
# check NaN (nice!)
vid_df.isna().sum()

gender                       0
school                       0
grade                        0
parent_edu                   0
ave_short_video_usage/day    0
exam_score                   0
dtype: int64

To facilitate data analysis, I will transform the binary variable into 0/1 instead of 1/2 as currently.

In [51]:
vid_df['gender'] = (vid_df['gender'] == 1).astype(int) # male = 1, others (female) = 0
vid_df['school'] = (vid_df['school'] == 1).astype(int) # urban = 1, others (suburban) = 0
vid_df.head(10)

,gender,school,grade,parent_edu,ave_short_video_usage/day,exam_score
0,1,1,2,6,3,276.0
1,1,1,2,5,1,295.0
2,0,1,2,5,5,289.0
3,0,1,5,5,4,244.0
4,1,1,2,5,3,279.0
5,1,1,4,4,3,288.0
6,1,1,4,5,1,283.0
7,1,1,4,4,3,275.0
8,1,1,4,4,3,278.0
9,0,1,4,5,2,282.5


In [52]:
# Save processed data to a file and import it
# index can change when resort/ filter row/ reset index -> set index=False to prevent inconsistencies
vid_df.to_csv("preprocessed_data.csv", index=False) 
vid_df

,gender,school,grade,parent_edu,ave_short_video_usage/day,exam_score
0,1,1,2,6,3,276.0
1,1,1,2,5,1,295.0
2,0,1,2,5,5,289.0
3,0,1,5,5,4,244.0
4,1,1,2,5,3,279.0
...,...,...,...,...,...,...
1047,0,0,2,4,3,197.5
1048,0,0,2,5,3,203.0
1049,0,0,1,6,2,291.0
1050,1,0,3,4,4,300.0
